# CMIC-DRC Workshop - Creating Processing Pipelines Using Nipype

## Example PET SUVR analysis workflow

Will Coath, 12th March 2021


## Overview

We have 5 participants with T1-weighted MRI and amyloid-PET (florbetapir/AV45) images. We want to compute a standard uptake value ratio (SUVR) for these participants. To calculate SUVR we will divide the PET tracer retention in a cortical region of interest by the retention in the cerebellum. 

We will construct a pipeline using many common software packages.

This is a workflow to demonstrate some of the basic aspects of Nipype, not necessarily the optimal analysis pipeline.

1. Co-register the 55-60 min PET frame to the 50-55 min frame and average them (FSL FLIRT and fslmaths)
2. Smooth PET wtih 6mm FWHM kernel (SPM)
3. Co-register MRI to MNI template (FSL FLIRT)
4. Co-register the PET to the MRI (NiftyReg)
5. Spatially normalise to MNI space template, then apply parameters to PET (SPM)
6. Normalise the PET uptake to cerebellum region (fslmaths)
7. Extract SUVR from cortical region (fslstats)

## Setting some parameters

Setting these at the top makes it easy to make changes later.

In [1]:
# Set some paths
experiment_dir = '/home/neuro/output'
data_dir = '/home/neuro/data/oasis_dataset'
result_dir='/home/neuro/output/results'
working_dir = '/home/neuro/output/working'

# list of subject ids
#subject_list = ['sub-OAS30001','sub-OAS30002','sub-OAS30003','sub-OAS30004','sub-OAS30005']


#just run on two for now
subject_list = ['sub-OAS30001','sub-OAS30002']

# Smoothing widths used during preprocessing of PET
fwhm = [6,6,6]

# Template to normalize to
template = '/home/neuro/data/avg152T1.nii'

# Cerebellum reference mask in MNI space
cereb_mask = '/home/neuro/data/voi_WhlCbl_2mm.nii'

# Cortical target region mask in MNI space
ctx_mask = '/home/neuro/data/voi_ctx_2mm.nii'

## Interfaces link to software packages

Nipype has interfaces with many common neuroimaging packages.

Let's import some interfaces we will use.

In [2]:
#import modules
from nipype.interfaces import fsl # FSL interfaces 
from nipype.interfaces import spm  # SPM interfaces
from nipype.interfaces import niftyreg # NiftyReg
from nipype.algorithms.misc import Gunzip # Gunzip
import nipype.interfaces.utility as util  # utility, for IdentityInterface and Function

## Interfaces go inside Nodes

We can package up specific instances of interfaces in Nodes, this makes them compatable with a pipeline.

In [3]:
import nipype.pipeline.engine as pe  # for Workflow and Node wrappers

### NODES TO PREPROCESS THE PET ###

# co-register last frame (55-60mins) to first (50-55mins) 
# tell FSL to save output as '.nii', as SPM won't accept '.nii.gz'
coreg_frames = pe.Node(interface=fsl.FLIRT(cost_func='normcorr',
                                        dof=6,
                                        output_type = "NIFTI"),
                    name='flt_coreg_frames')


# add frames together, takes in_file and operand_file and adds them
add_frames = pe.Node(fsl.BinaryMaths(operation='add',
                                     output_type = "NIFTI"),
                         name='fmaths_add_frames')

# divide by 2 to make 50-60min average frame
div_frames = pe.Node(fsl.BinaryMaths(operation='div',
                                     operand_value=2,
                                     output_type = "NIFTI"),
                         name='fmaths_div_frames')

# Node containing spm interface to smooth the 50-60 PET image by 6mm fwhm 
smooth = pe.Node(interface=spm.Smooth(fwhm=fwhm), 
                   name='spm_smooth')



## Connecting nodes together in a Workflow

Here we start a Workflow to connect the Nodes together, we take outputs from one Node and pass them to another.

You can find out what the input and output of certain interfaces are called here:


In [4]:
suvr_pipeline = pe.Workflow(name='suvr')

suvr_pipeline.base_dir = working_dir

suvr_pipeline.connect([
    # give co-registered second frame from fslmaths output to sum
    # then to divide by 2
    (coreg_frames, add_frames, [('out_file', 'operand_file')]),
    (add_frames, div_frames, [('out_file', 'in_file')]),
    (div_frames, smooth, [('out_file', 'in_files')]),
])

## More nodes...

Next, we will create some Nodes to spatially normalise the PET using SPM normalise.

In [5]:

coreg_template = pe.Node(interface=fsl.FLIRT(cost_func='mutualinfo',
                                             reference=template,
                                             dof=6,
                                             output_type = "NIFTI"),
                    name='flt_coreg_template')

coreg_petmr = pe.Node(interface=niftyreg.RegAladin(rig_only_flag=True),
                    name='aladin_coreg_petmr')

# bounding box needs to be set to NaN in SPM, create nan float variable
n=float("nan")

# spatially normalise the T1 to the MNI template - Estimate & Write (estwrite) mode is the default
norm_anat = pe.Node(interface=spm.Normalize(template=template,
                                            write_bounding_box=[[n, n, n], [n, n, n]]),
                    name='spm_norm_anat')

# to apply the non-linear transform from 'norm_anat' to the PET (write only mode)
norm_pet = pe.Node(interface=spm.Normalize(jobtype='write',
                                          write_bounding_box=[[n, n, n], [n, n, n]]),
                    name='spm_norm_pet')

# node to unzip the PET file before SPM step, as RegAladin has saved resampled PET file as '.nii.gz'
gunzip_pet = pe.Node(Gunzip(),
                name='gunzip_pet')

And we can connect these to the same pipeline

In [ ]:
suvr_pipeline.connect([
    # pass PET from smoothing to niftyReg
    (smooth, coreg_petmr, [('smoothed_files', 'flo_file')]),
    (coreg_template, coreg_petmr, [('out_file', 'ref_file')]),
    (coreg_template, norm_anat, [('out_file', 'source')]),
    (coreg_petmr, gunzip_pet, [('res_file', 'in_file')]),
    (gunzip_pet, norm_pet, [('out_file', 'apply_to_files')]),
    (norm_anat, norm_pet, [('normalization_parameters', 'parameter_file')]),
])


## ...and speficy some more nodes...and connect them to the pipeline

To calculate SUVR and extract mean values.

In [6]:
# extract reference region mean raw uptake
ref_uptake = pe.Node(fsl.ImageStats(mask_file=cereb_mask,
                                   op_string='-k %s -M'),
                name='fstats_ref_uptake')

# extract target region mean raw uptake (just for possible use later in models)
tar_uptake = pe.Node(fsl.ImageStats(mask_file=ctx_mask,
                                   op_string='-k %s -M'),
                name='fstats_tar_uptake')

# create SUVR image by dividing PET MNI image by result of ref_uptake
suvr_img = pe.Node(fsl.BinaryMaths(operation='div',
                                     output_type = "NIFTI_GZ"),
                name='fmaths_suvr_img')

#extract mean SUVR value from target region
tar_suvr = pe.Node(fsl.ImageStats(mask_file=ctx_mask,
                                   op_string='-k %s -M'),
                name='fstats_tar_suvr')

suvr_pipeline.connect([
    # connect PET in MNI space to fslstats to get reference mean uptake
    (norm_pet, ref_uptake, [('normalized_files', 'in_file')]),
    # connect PET in MNI space to fslstats to get target mean uptake
    (norm_pet, tar_uptake, [('normalized_files', 'in_file')]),
    # give MNI PET to fslmaths to calculate SUVR
    (norm_pet, suvr_img, [('normalized_files', 'in_file')]),
    # give output stat, the mean of cerebellum uptake, to fslmaths to 
    # divide the pet image by it
    (ref_uptake, suvr_img, [('out_stat', 'operand_value')]),
    # give SUVR image to 
    (suvr_img, tar_suvr, [('out_file', 'in_file')]),
])

## Creating our own function 

If you need to do anything else you can define a function and wrap it inside a Node.

In [7]:
#create custom function for writing output to csv file
def OutCSV(ref_uptake,tar_uptake,tar_suvr):
    #note we need to import python modules within the function, as environment is separate
    import os 
    header = "reference_uptake,target_uptake,target_suvr \n"
    row = str(ref_uptake) + ',' + str(tar_uptake) + ',' + str(tar_suvr)
    
    file_name = 'result.csv'
    
    with open(file_name, 'w') as fp:
        fp.write(header)
        fp.write(row)
    return os.path.abspath(file_name)


Then we need to package our function inside a Node, in order to connect it to the Workflow

In [ ]:
#define Node with our function inside to put it into our workflow
get_csv = pe.Node(util.Function(input_names=['ref_uptake','tar_uptake', 'tar_suvr'],
                                output_names=['result_file'],
                                function=OutCSV),
                  name='get_csv')


suvr_pipeline.connect([
    (ref_uptake, get_csv, [('out_stat', 'ref_uptake')]),
    (tar_uptake, get_csv, [('out_stat', 'tar_uptake')]),
    (tar_suvr, get_csv, [('out_stat', 'tar_suvr')]), 
])

## Grab input data

Hang on a minute...how do the Nodes know how to find our scans?

We create a node called 'infosource' containing the 'IdentityInterface', this iterates over our subject list (specified at the top) and feeds them into the 'datasource' Node, containing a 'DataGrabber' interface. The DataGrabber uses a template and fills in the subject ids provided to pick up the scans we want. 

The 'info' variable is a dictionary that links each id with both PET frames and the T1 image. The DataGrabber then uses a template to construct paths to each image.


In [8]:
import nipype.interfaces.io as nio  # Data input and output, DataGrabber and DataSink

infosource = pe.Node(
    interface=util.IdentityInterface(fields=['subject_id']),
    name="infosource")

infosource.iterables = ('subject_id', subject_list)

# Map field names to individual subject runs.
info = dict(
    pet1=[['subject_id','pet','t50-55']],
    pet2=[['subject_id','pet','t55-60']],
    anat=[['subject_id', 'anat','T1w']])

datasource = pe.Node(interface=nio.DataGrabber(infields=['subject_id'], 
                                               outfields=['pet1', 'pet2','anat']),
                     name='datasource')

datasource.inputs.base_directory = data_dir
datasource.inputs.template = '%s/ses-test/%s/*%s*.nii.gz'
datasource.inputs.template_args = info
datasource.inputs.sort_filelist = True

## Connect inputs to SUVR Workflow


In [9]:
suvr_pipeline.connect([
    # connect subject id list to DataGrabber
    (infosource, datasource, [('subject_id', 'subject_id')]),
    # give both PET frames (pet1 and pet2) to FLIRT to co-register second frame to first 
    (datasource, coreg_frames, [('pet2', 'in_file'),
                               ('pet1','reference')]),
    # give first pet frame to fslmaths 
    (datasource, add_frames, [('pet1', 'in_file')]),
    # pass T1 MRI to node to co-reg with template
    (datasource, coreg_template, [('anat', 'in_file')]), 
])

## Store the output we want

DataSink interface

In [10]:
datasink = pe.Node(interface=nio.DataSink(), name="datasink")

datasink.inputs.base_directory = experiment_dir
datasink.inputs.container = result_dir

## Define substitution strings
substitutions = [('_subject_id_', ''),
                 ('wssub-','mni_smoothed_sub-'),
                 ("_t50-55_maths_maths", "_t50-60-avg"),
                 ('ssub-', 'smoothed_sub-'),
                 ('avg_res_maths.nii','avg_suvr.nii')
                ]

## Feed the substitution strings to the DataSink node
datasink.inputs.substitutions = substitutions

suvr_pipeline.connect([
    #connect the list of participants to datasink to make separate containers for each
    #(infosource, datasink, [('subject_id', 'container')]),
    #save output from PET frame co-reg in datasink
    (coreg_frames, datasink, [('out_file', 'coreg_frames'),
                             ('out_matrix_file', 'coreg_frames.@xfm')]),
    #save average 50-60min frame image in the datasink
    (div_frames, datasink, [('out_file', 'pet_t5060')]),
    #save smoothed average 50-60min frame
    (smooth, datasink, [('smoothed_files', 'pet_t5060.@smoothed')]),
    #save PET file that has been co-registered to the MRI
    (coreg_petmr, datasink, [('res_file', 'coreg_pet-t1')]),
    #and save the PET to T1 transformation matrix in xfm sub dir
    (coreg_petmr, datasink, [('aff_file', 'coreg_pet-t1.@xfm')]),
    #save spatially normalised T1 image to anat_mni 
    (norm_anat, datasink, [('normalized_source', 't1_mni')]),
    # and save normalisation parameters to sub directory
    (norm_anat, datasink, [('normalization_parameters', 't1_mni.@xfm')]),
    #save spatially normalised PET image to pet_mni
    (norm_pet, datasink, [('normalized_files', 'pet_mni')]),
    #add suvr image to datasink
    (suvr_img, datasink, [('out_file','suvr_result')]),
    (get_csv, datasink, [('result_file','suvr_result.@csv')]),
])


## Run Workflow and print graph

In [11]:
suvr_pipeline.run(plugin='MultiProc') #run in parallel
suvr_pipeline.write_graph(dotfilename='/home/neuro/output/results/workflow_graph.dot')

210311-11:13:44,822 nipype.workflow INFO:
	 Workflow suvr settings: ['check', 'execution', 'logging', 'monitoring']
210311-11:13:45,27 nipype.workflow INFO:
	 Running in parallel.
210311-11:13:45,62 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 5 jobs ready. Free memory (GB): 1.75/1.75, Free processors: 2/2.
210311-11:13:45,207 nipype.workflow INFO:
	 [Node] Setting-up "suvr.datasource" in "/home/neuro/suvr/_subject_id_sub-OAS30005/datasource".
210311-11:13:45,209 nipype.workflow INFO:
	 [Node] Setting-up "suvr.datasource" in "/home/neuro/suvr/_subject_id_sub-OAS30004/datasource".210311-11:13:45,244 nipype.workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")

210311-11:13:45,284 nipype.workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
210311-11:13:45,406 nipype.workflow INFO:
	 [Node] Finished "suvr.datasource".
210311-11:13:45,420 nipype.workflow INFO:
	 [Node] Finished "suvr.datasource".
210311-11:13:47,58 nipype

PermissionError: [Errno 13] Permission denied: '/home/neuro/output/results'

## Summary

### Interface:
specific software (e.g. FSL, SPM, FreeSurfer, ANTs ...) are wrapped in interfaces, within a node instances of an interface can be run

### Node:
Holds an interfact, you can give it a name, a takes input and gives output

### Workflow:
Connected series of nodes, can be run parallel and or sequential

### Pipeline:
Pretty much the same as a workflow

### Modules:
For each interface the according modules have to be imported in the usual pythonic manner

### Crash Files:

In bash command line: 
nipypecli crash path/to/crash/file.pklz
